# Setup

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2 as cv
from PIL import Image
import re
import glob
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf
import math

plt.ion()   # interactive mode

BOTAR AQUI SUA PASTA DO NOTEBOOK

In [2]:
# pasta_do_notebook = '/content/drive/MyDrive/Blur_CLas'
pasta_do_notebook = '.'
os.chdir(pasta_do_notebook)

In [3]:
os.getcwd()

'd:\\daniel_moreira\\reconhecimento_de_padroes\\reconhecimento_de_padroes\\noisedetect'

# Dataset

In [4]:
# Function to find reference and noisy images in a folder
def find_images_in_folder(folder):
    reference_image = None
    noisy_image = None

    # Iterate through files in the folder
    for filename in os.listdir(folder):
        if filename.lower().endswith(".jpg"):
            file_path = os.path.join(folder, filename)

            # Check if the image contains "Reference" or "Noisy"
            if "reference" in filename.lower():
                reference_image = cv.imread(file_path)
            elif "noisy" in filename.lower():
                noisy_image = cv.imread(file_path)

            # Break if both images are found
            if reference_image is not None and noisy_image is not None:
                break

    return reference_image, noisy_image

# Main function to process folders
def read_renoir(root_folders, num_images=0):
    print('Reading RENOIR...')
    targets, labels = [], []
    for root_folder in os.listdir(root_folders):
        root_folder_path = os.path.join(root_folders, root_folder)
        print('Reading folder: ', root_folder)
        for foldername in os.listdir(root_folder_path):
            folder_path = os.path.join(root_folder_path, foldername)
            if os.path.isdir(folder_path):
                reference_image, noisy_image = find_images_in_folder(folder_path)
                
                targets.append(noisy_image)
                labels.append(reference_image)
                if num_images > 0 and len(targets) % num_images == 0:
                    break
    print('Reading complete.')
    y = np.ones(len(targets))
    np.concatenate((y, np.zeros(len(labels))))
    np.concatenate((targets, labels))
    return targets, y

In [5]:
train_path = 'datasets/train'
val_path = 'datasets/val'
test_path = 'datasets/test'

In [6]:
X_train, y_train = read_renoir(train_path, num_images=0)
X_val, y_val = read_renoir(val_path, num_images=0)
X_test, y_test = read_renoir(test_path, num_images=0)

Reading RENOIR...
Reading folder:  Mi3_Aligned
Reading folder:  S90_Aligned
Reading folder:  T3i_Aligned
Reading complete.
Reading RENOIR...
Reading folder:  Mi3_Aligned


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Reading folder:  S90_Aligned
Reading folder:  T3i_Aligned
Reading complete.
Reading RENOIR...
Reading folder:  Mi3_Aligned
Reading folder:  S90_Aligned
Reading folder:  T3i_Aligned
Reading complete.


In [7]:
print(len(X_train))
print(len(X_test))
print(len(X_val))
print(X_val[:5])
print(len(y_val))
print(y_val[:5])

72
24
24
[array([[[ 22,  64,  69],
        [ 17,  61,  68],
        [ 18,  63,  74],
        ...,
        [ 19,  37,  38],
        [ 14,  38,  36],
        [ 18,  48,  43]],

       [[ 29,  65,  71],
        [ 23,  60,  68],
        [ 23,  61,  73],
        ...,
        [ 14,  28,  34],
        [ 15,  36,  37],
        [ 12,  36,  34]],

       [[ 36,  59,  67],
        [ 30,  55,  65],
        [ 32,  58,  70],
        ...,
        [ 16,  25,  39],
        [ 22,  32,  42],
        [ 21,  33,  39]],

       ...,

       [[ 34,  51,  77],
        [ 34,  55,  76],
        [ 31,  59,  66],
        ...,
        [ 54,  92, 127],
        [ 64, 100, 146],
        [ 59,  96, 148]],

       [[ 36,  65,  86],
        [ 27,  61,  74],
        [ 19,  59,  58],
        ...,
        [ 57, 105, 129],
        [ 60, 111, 144],
        [ 64, 116, 153]],

       [[ 40,  76,  94],
        [ 28,  67,  76],
        [ 21,  67,  61],
        ...,
        [ 54, 107, 127],
        [ 53, 110, 136],
        [ 52, 

In [8]:
# Function to create patches from an image with a specified patch size
def split_image(image, patch_size=256, qtd=1):

    height, width, _ = image.shape

    # Calculate the number of rows and columns needed for patches
    num_rows = math.ceil(height / patch_size)
    num_cols = math.ceil(width / patch_size)

    out_height = patch_size * num_rows
    out_width = patch_size * num_cols

    # Calculate the amount of padding needed
    pad_height = out_height - height
    pad_width = out_width - width

    # Pad the image with zeros if necessary
    if pad_height > 0 or pad_width > 0:
        image = cv.copyMakeBorder(image, 0, pad_height, 0, pad_width, cv.BORDER_CONSTANT, value=0)

    # Initialize an empty list to store the patches
    patches = []

    # Iterate through the image and extract patches
    for y in range(0, out_height, patch_size):
        for x in range(0, out_width, patch_size):
            patch = image[y:y+patch_size, x:x+patch_size]
            patches.append(patch)

    if qtd < 1 and qtd > 0:
        np.random.shuffle(patches)
        patches = patches[:int(len(patches)*qtd)]

    return np.array(patches)

In [9]:
# import numpy as np
# from PIL import Image

# # Define your NumPy array of images
# image_array = ex_patches

# # Loop through each image in the array
# for i in range(len(image_array)):
#     # Convert the NumPy array to a PIL image
#     image = Image.fromarray(image_array[i])

#     # Display the image in the Jupyter notebook
#     display(image)

In [10]:
def patchify(X, y, qtd_patches=1):
    X_patches, y_patches = [], []
    for i in range(len(X)):
        patches = split_image(X[i], 256, qtd_patches)
        X_patches.extend(patches)
        for patch in patches:
            if y[i] == 1:
                y_patches.append(1)
            else:
                y_patches.append(0)
    return np.array(X_patches), np.array(y_patches)

In [11]:
X_train, y_train = patchify(X_train, y_train, 0.2)
X_test, y_test = patchify(X_test, y_test, 0.2)
X_val, y_val = patchify(X_val, y_val, 0.2)

In [12]:
if not os.path.exists('datasets'):
    os.makedirs('datasets')

np.save('datasets/X_train.npy', X_train)
np.save('datasets/y_train.npy', y_train)

np.save('datasets/X_test.npy', X_test)
np.save('datasets/y_test.npy', y_test)

np.save('datasets/X_val.npy', X_val)
np.save('datasets/y_val.npy', y_val)

In [13]:
print(len(X_train))
print(len(X_test))
print(len(X_val))

2700
928
915


# Treino

In [14]:
X_train_patches = np.load('datasets/X_train.npy')
y_train_patches = np.load('datasets/y_train.npy')

X_val_patches = np.load('datasets/X_val.npy')
y_val_patches = np.load('datasets/y_val.npy')

In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint

if not os.path.exists('weights'):
    os.mkdir('weights')

checkpoint = ModelCheckpoint(
    filepath='weightscheckpoint_path.h5',
    monitor='val_loss',  # Métrica para monitorar (pode ser 'val_loss' para perda de validação)
    save_best_only=True,  # Salva apenas o melhor modelo
    mode='min',  # 'max' se a métrica monitorada deve ser maximizada, 'min' se deve ser minimizada
    verbose=1  # Exibe mensagens durante o salvamento do modelo
)

In [17]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import certifi

model = tf.keras.applications.densenet.DenseNet121(
    include_top=False,
    weights='imagenet',
    input_shape=(256, 256, 3)
)

top = GlobalAveragePooling2D()(model.output)
top = Dense(1024, activation='relu')(top)
top = Dense(1, activation='sigmoid')(top)
model = Model(inputs=model.input, outputs=top)

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/densenet/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5: None -- unknown url type: https

In [ ]:
from tensorflow.keras.applications.densenet import preprocess_input
def preprocess_data(X, y, shuffle=False):
    # Shuffle the data if specified
    X = X/255
    if shuffle:
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        X = X[indices]
        y = y[indices]

    return X, y

In [ ]:
X_train, y_train = preprocess_data(X_train, y_train)
X_test, y_test = preprocess_data(X_test, y_test)
X_val, y_val = preprocess_data(X_val, y_val)
print(X_train.shape, X_test.shape, X_val.shape)

(1482, 256, 256, 3) (213, 256, 256, 3) (204, 256, 256, 3)


In [ ]:
# Convert NumPy arrays to TensorFlow tensors
X_train = tf.convert_to_tensor(X_train)
y_train = tf.convert_to_tensor(y_train)

# Create a tf.data.Dataset
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

# Batch, shuffle, and repeat the dataset
batch_size = 16
dataset = dataset.shuffle(buffer_size=len(X_train))
dataset = dataset.batch(batch_size)
# dataset = dataset.repeat()

In [ ]:
# Compile o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treine o modelo
# r = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[checkpoint], batch_size = 8)
r = model.fit(dataset, validation_data=(X_val,y_val), epochs=100, callbacks=[checkpoint])

# Avalie o modelo no conjunto de teste
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Exiba as métricas calculadas
print("Loss:", test_loss)
print("Accuracy:", test_accuracy)
print("Validation", r.history['val_loss'])

Epoch 1/50
93/93 [==============================] - ETA: 0s - loss: 0.6717 - accuracy: 0.7274
Epoch 1: val_loss improved from inf to 0.68267, saving model to checkpoint_path.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


93/93 [==============================] - 109s 386ms/step - loss: 0.6717 - accuracy: 0.7274 - val_loss: 0.6827 - val_accuracy: 0.7059
Epoch 2/50
93/93 [==============================] - ETA: 0s - loss: 0.5322 - accuracy: 0.7713
Epoch 2: val_loss did not improve from 0.68267
93/93 [==============================] - 23s 243ms/step - loss: 0.5322 - accuracy: 0.7713 - val_loss: 0.6997 - val_accuracy: 0.7255
Epoch 3/50
93/93 [==============================] - ETA: 0s - loss: 0.5241 - accuracy: 0.7632
Epoch 3: val_loss did not improve from 0.68267
93/93 [==============================] - 23s 245ms/step - loss: 0.5241 - accuracy: 0.7632 - val_loss: 2.0463 - val_accuracy: 0.6176
Epoch 4/50
93/93 [==============================] - ETA: 0s - loss: 0.4708 - accuracy: 0.7989
Epoch 4: val_loss improved from 0.68267 to 0.54985, saving model to checkpoint_path.h5
93/93 [==============================] - 26s 284ms/step - loss: 0.4708 - accuracy: 0.7989 - val_loss: 0.5499 - val_accuracy: 0.7696
Epoch 5/

In [ ]:
loss = r.history['loss']
val_loss = r.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc1 = r.history['accuracy']
val_acc1 = r.history['val_accuracy']
plt.plot(epochs, acc1, 'y', label='Training acurácia')
plt.plot(epochs, val_acc1, 'r', label='Validation acurácia')
plt.title('Training and validation acurácia')
plt.xlabel('Epochs')
plt.ylabel('acurácia')
plt.legend()
plt.show()

# Teste

In [ ]:
X_test_patches = np.load('datasets/X_test.npy')
y_test_patches = np.load('datasets/y_test.npy')

In [ ]:
print(len(X_test))

213


In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model

model = tf.keras.applications.densenet.DenseNet121(
    include_top=False,
    weights=None,
    input_shape=(256, 256, 3)
)

top = GlobalAveragePooling2D()(model.output)
top = Dense(1024, activation='relu')(top)
top = Dense(1, activation='sigmoid')(top)
model = Model(inputs=model.input, outputs=top)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

from tensorflow.keras.applications.densenet import preprocess_input
def preprocess_data(X, y, shuffle=False):
    # Shuffle the data if specified
    X = X/255
    if shuffle:
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        X = X[indices]
        y = y[indices]

    return X, y

In [ ]:
X_test, y_test = preprocess_data(X_test, y_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model.load_weights(os.path.join(pasta_do_notebook, 'weights', 'checkpoint_path.h5'))

y_pred = model.predict(X_test)

y_pred_classes = np.round(y_pred)

accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes)
recall = recall_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.6667
Precision: 0.6667
Recall: 1.0000
F1 Score: 0.8000
